# Application of Named Entity Recognition tools on Funding Information

A code implemented to extract named entites from a text. NER tools are used for extracting organization names from funding information obtained from PubMed research papers

# 1. Setting up the environment


In [1]:
# Connecting the google colab noteboook to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import timeit
t_0 = timeit.default_timer()

In [3]:
# importing required libraries
import pandas as pd
import numpy as np
import spacy
import pickle

In [4]:
# importing the dataset
ack_df = pd.read_csv("/content/gdrive/MyDrive/Summer_Research_2022/Data/cumulative_ack_data.csv")
ack_df.columns = ['Article_Title', 'PMC_ID', 'DOI', 'acknowledgement']
ack_df.head()

,Article_Title,PMC_ID,DOI,acknowledgement
0,Impact of antibiotics on the human microbiome ...,8756738,na,The authors were funded in part by Science Fou...
1,Novel nitrite reductase domain structure sugge...,8756737,na,The authors thank Dr. Ranjani Murali for advic...
2,Efficacy of antifungal agents against fungal s...,8756736,na,This study was funded by Dr. Pfleger Arzneimit...
3,Assembly and comparative analysis of the first...,8756732,10.1186/s12870-021-03416-5,Thanks to all the members of the Institute of ...
4,Characterizing the effects of different chemic...,8756731,10.1186/s13007-021-00835-1,Not applicable. This work was supported by the...


In [5]:
# removing the rows for which there is no funding information
ack_df = ack_df[ack_df['acknowledgement'] != 'na']

# 2. Applying NER on funding information

## 1. Spacy EntityRecognizer (en_core_web_sm)

ner categories - LOC, MISC, ORG, PER


In [6]:
nlp = spacy.load('en_core_web_sm')

# Test sentence
doc = nlp('Apple is looking at buying U.K. startup for $1 billion')
for ent in doc.ents:
  print(ent,ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY


In [ ]:
# applying the EntityRecognizer to the acknowledgement dataset
ack_ner = []
for ele in ack_df['acknowledgement']:
  ele = nlp(ele).ents
  temp = {}
  for ent in ele:
    if ent.label_ in temp:
      temp[ent.label_].append(ent.text)
    else:
      temp[ent.label_] = [ent.text]
  
  ack_ner.append(temp)
  
ack_df['NER_Spacy (en_core_web_sm)'] = ack_ner
ack_df.reset_index(inplace= True)

In [ ]:
ack_df.head()

In [ ]:
print(ack_df.loc[4,:]['acknowledgement'])
print(ack_df.loc[4,:]['NER_Spacy (en_core_web_sm)'])

## 2. Spacy EntityRecognizer (en_core_web_md)

ner categories - CARDINAL, DATE, EVENT, FAC, GPE, LANGUAGE, LAW, LOC, MONEY, NORP, ORDINAL, ORG, PERCENT, PERSON, PRODUCT, QUANTITY, TIME, WORK_OF_ART

In [ ]:
# https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab
import spacy.cli
spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

# Test sentence
doc = nlp('Apple is looking at buying U.K. startup for $1 billion')
for ent in doc.ents:
  print(ent,ent.label_)

In [ ]:
# applying the EntityRecognizer to the acknowledgement dataset
ack_ner = []
for ele in ack_df['acknowledgement']:
  ele = nlp(ele).ents
  temp = {}
  for ent in ele:
    if ent.label_ in temp:
      temp[ent.label_].append(ent.text)
    else:
      temp[ent.label_] = [ent.text]
  
  ack_ner.append(temp)
  
ack_df['NER_Spacy (en_core_web_md)'] = ack_ner

In [ ]:
ack_df.head()

In [ ]:
print(ack_df.loc[1,:]['acknowledgement'])
print(ack_df.loc[1,:]['NER_Spacy (en_core_web_md)'])

## 3. Spacy EntityRecognizer (en_core_web_lg)

ner categories  - CARDINAL, DATE, EVENT, FAC, GPE, LANGUAGE, LAW, LOC, MONEY, NORP, ORDINAL, ORG, PERCENT, PERSON, PRODUCT, QUANTITY, TIME, WORK_OF_ART

In [ ]:
# https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

# Test sentence
doc = nlp('Apple is looking at buying U.K. startup for $1 billion')
for ent in doc.ents:
  print(ent,ent.label_)

In [ ]:
# applying the EntityRecognizer to the acknowledgement dataset
ack_ner = []
for ele in ack_df['acknowledgement']:
  ele = nlp(ele).ents
  temp = {}
  for ent in ele:
    if ent.label_ in temp:
      temp[ent.label_].append(ent.text)
    else:
      temp[ent.label_] = [ent.text]
  
  ack_ner.append(temp)
  
ack_df['NER_Spacy (en_core_web_lg)'] = ack_ner

In [ ]:
ack_df.head()

In [ ]:
print(ack_df.loc[1,:]['acknowledgement'])
print(ack_df.loc[1,:]['NER_Spacy (en_core_web_lg)'])

4. Flair - English NER in Flair (large model) - 4 class model

## 4. Flair - English NER in Flair (large model)

4 class model

In [ ]:
!pip install flair

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
# Make a sentence
sentence = Sentence("Apple is looking at buying U.K. startup for $1 billion")

# Load the NER tagger
# This file is around 1.5 GB so will take a little while to load.
tagger = SequenceTagger.load("flair/ner-english-large")

# Run NER over sentence
tagger.predict(sentence)

In [ ]:
# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
# applying the Flair to the acknowledgement dataset
ack_ner = []
for ele in ack_df['acknowledgement']:
  sentence = Sentence(ele)
  tagger.predict(sentence)
  ner_sent = sentence.get_spans('ner')
  temp = {}
  for ent in ner_sent:
    ent_label = ent.get_label("ner").value
    ent_text = ent.text

    if ent_label in temp:
      temp[ent_label].append(ent_text)
    else:
      temp[ent_label] = [ent_text]
  
  ack_ner.append(temp)
  

ack_df['NER_Flair'] = ack_ner

In [ ]:
ack_df.head()

In [ ]:
print(ack_df.loc[1,:]['acknowledgement'])
print(ack_df.loc[1,:]['NER_Flair'])

# Output Data generation

In [ ]:
# extracting the organization entity from NER_Spacy (en_core_web_sm)
ner_org = []
for ele in ack_df['NER_Spacy (en_core_web_sm)']:
  temp = []
  if len(ele) != 0 and 'ORG' in ele.keys():
    temp = ele['ORG']
    ner_org.append(temp)
  else:
    ner_org.append("NA")

ack_df['NER_spacy_sm_org'] = ner_org

# extracting the organization entity from NER_Spacy (en_core_web_md)
ner_org = []
for ele in ack_df['NER_Spacy (en_core_web_md)']:
  temp = []
  if len(ele) != 0 and 'ORG' in ele.keys():
    temp = ele['ORG']
    ner_org.append(temp)
  else:
    ner_org.append("NA")

ack_df['NER_spacy_md_org'] = ner_org
    
# extracting the organization entity from NER_Spacy (en_core_web_lg)
ner_org = []
for ele in ack_df['NER_Spacy (en_core_web_lg)']:
  temp = []
  if len(ele) != 0 and 'ORG' in ele.keys():
    temp = ele['ORG']
    ner_org.append(temp)
  else:
    ner_org.append("NA")

ack_df['NER_spacy_lg_org'] = ner_org


# extracting the organization entity from NER_Flair
ner_org = []
for ele in ack_df['NER_Flair']:
  temp = []
  if len(ele) != 0 and 'ORG' in ele.keys():
    temp = ele['ORG']
    ner_org.append(temp)
  else:
    ner_org.append("NA")

ack_df['NER_Flair_org'] = ner_org

In [ ]:
ack_df

In [ ]:
# saving the file to csv
ack_df.to_csv("/content/ack_ner.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
with open('drive/MyDrive/ack_ner.pickle', 'wb') as handle:
    pickle.dump(ack_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
t_1 = timeit.default_timer()
print("The time elapsed: ", t_1 - t_0)